# 模型架構與訓練
負責切分訓練集、正規化、訓練模型 ，以預測 **0056 (元大高股息)** 的未來收盤價。
1. **資料處理**: 載入多檔相關股票數據，並進行合併與清洗。
2. **特徵工程**: 計算 MA, RSI, MACD 等關鍵技術指標。
3. **模型訓練**: 結合 XGBoost, Random Forest, LightGBM 三大模型進行投票決策。
4. **成效評估**: 使用 RMSE 與 MAPE 驗證模型準確度。

In [1]:
# ========================================================
# 1. 核心數據處理套件 (Core Data Processing)
# ========================================================
import pandas as pd  # 資料分析神器：用來操作 DataFrame，就像 Python 裡的 Excel
import numpy as np   # 數學運算基石：處理陣列、矩陣運算，速度比 Python 內建列表快

# ========================================================
# 2. 視覺化繪圖套件 (Visualization)
# ========================================================
import matplotlib.pyplot as plt # 基礎繪圖庫：畫折線圖、設定圖表大小與軸標籤
import seaborn as sns           # 進階美化庫：基於 Matplotlib，畫熱力圖更漂亮，風格更現代

# 設定 seaborn 的預設美學 (讓圖表自動變漂亮)
sns.set_style("whitegrid")      # 背景加格子
sns.set_context("notebook")     # 字體設為適合閱讀的大小

# ========================================================
# 3. 模型評估指標 (Evaluation Metrics)
# ========================================================
# 用來幫模型打分數的工具
# mean_squared_error: 計算 RMSE (誤差金額) 用
# mean_absolute_percentage_error: 計算 MAPE (誤差百分比) 用
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# ========================================================
# 4. 自定義模組 (Project Modules)
# ========================================================
# 這是我們專案自己寫的三個核心檔案
import data_loader          # 步驟 1: 搬運工 (讀取資料)
import data_preprocessing   # 步驟 2: 清潔工 (洗資料)
import feature_eng          # 步驟 3: 工程師 (算技術指標)

## 1. 資料準備 (Data Preparation)

在此階段，我們呼叫自定義的模組來執行標準化的資料處理流程 (ETL)：
* **Load**: 讀取所有股票 CSV。
* **Clean**: 處理缺值與格式問題。
* **Feature Engineering**: 自動計算技術指標。

In [2]:
# 重新跑一次流程拿到資料
df_raw = data_loader.load_and_merge_data()
df_clean = data_preprocessing.clean_data(df_raw)
df_features = feature_eng.add_technical_indicators(df_clean)

print("資料準備完成，形狀:", df_features.shape)

🚀 [Loader] 開始讀取資料...
✅ 資料載入完成 (尚未清洗)！大小：(852, 12)
🧹 [Preprocessing] 開始清洗資料...
✅ 資料清洗完成！沒有 NaN 了。
📊 [Feature Engineering] 偵測到 11 支股票，開始計算全套技術指標...
✅ 特徵工程完成！目前的欄位數: 122
資料準備完成，形狀: (852, 122)


/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_Vol_20'] = df[col].pct_change().rolling(window=20).std()
/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_BBP'] = (df[col] - lower_band) / (upper_band - lower_band + 1e-9)
/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

## 2. 資料集切分 (Train-Test Split)

為了模擬真實的時間序列預測場景，我們採用 **「時間序列切分法」**，而非隨機切分：
* **訓練集 (Train)**: 使用過去的歷史數據。
* **驗證集 (Validation)**: 保留最後 30 天的數據作為模擬考題，嚴格禁止模型在訓練時看到這些答案。

In [3]:
# 切分訓練集與驗證集 (為了實驗，我們自己切)
# 我們拿最後 30 天當作驗證集 (模擬考)，前面當訓練集
test_days = 30
train_df = df_features.iloc[:-test_days]
val_df = df_features.iloc[-test_days:]

# 設定目標欄位名稱
TARGET_COL = '0056_close_y'

# 準備 X (特徵) 和 y (答案)
# 記得從 X 中拿掉答案欄位，以免發生 Data Leakage (作弊)
X_train = train_df.drop(columns=[TARGET_COL, 'date'], errors='ignore')
y_train = train_df[TARGET_COL]

X_val = val_df.drop(columns=[TARGET_COL, 'date'], errors='ignore')
y_val = val_df[TARGET_COL]

print(f"訓練集大小: {X_train.shape}, 驗證集大小: {X_val.shape}")

訓練集大小: (822, 120), 驗證集大小: (30, 120)


## 3. 模型訓練 (Model Training) - 夢幻隊伍

我們採用 **集成學習 (Ensemble Learning)** 策略，結合三種不同特性的模型進行投票，以提升預測的穩定性與準確度：
1. **XGBoost**: 梯度提升樹，擅長捕捉複雜非線性關係。
2. **Random Forest**: 隨機森林，抗雜訊能力強，不易過擬合。
3. **LightGBM**: 輕量級梯度提升，運算速度快且精度高。

In [4]:
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 1. 定義個別的專家 (模型)
# --- 專家 A: XGBoost ---
xgb = XGBRegressor(
    n_estimators=1000, 
    learning_rate=0.05, 
    max_depth=6,
    random_state=42,
    n_jobs=-1
)

# --- 專家 B: Random Forest (隨機森林) ---
rf = RandomForestRegressor(
    n_estimators=500,  # 樹的數量
    max_depth=10,      # 不要太深，避免死背
    random_state=42,
    n_jobs=-1
)

# --- 專家 C: LightGBM (另一個梯度提升樹) ---
lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
    verbose=-1 # 安靜模式
)

# 2. 組建夢幻隊伍 (投票器)
# weights 可以設定權重，例如讓 XGBoost 講話大聲一點
print("🤝 正在組建集成模型 (Ensemble Model)...")
model = VotingRegressor(
    estimators=[
        ('xgb', xgb), 
        ('rf', rf), 
        ('lgbm', lgbm)
    ],
    weights=[2, 1, 1] # 權重: XGB佔2份，其他各1份 (您可以自己調)
)

# 3. 訓練 (跟原本一模一樣)
print("🚀 開始訓練夢幻隊伍...")
model.fit(X_train, y_train)

# 4. 預測與評估
preds = model.predict(X_val)

# 算分數
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_val, preds)
print(f"📊 集成模型 MAPE: {mape:.2%}")

🧪 測試模式：切換回單一 XGBoost 模型，測試 Baseline...


## 4. 特徵重要性分析 (Feature Importance)

為了理解模型的決策邏輯，我們提取了各個子模型的特徵重要性並取平均。這能幫助我們回答：**「AI 覺得哪些指標最能預測股價？」**

In [5]:
# --- 綜合三位專家的意見 ---
plt.figure(figsize=(10, 6))

# 1. 抓出三位專家
xgb_imp = model.named_estimators_['xgb'].feature_importances_
rf_imp = model.named_estimators_['rf'].feature_importances_
lgbm_imp = model.named_estimators_['lgbm'].feature_importances_

# 2. 平均他們的意見
avg_importances = (xgb_imp + rf_imp + lgbm_imp) / 3
indices = np.argsort(avg_importances)[::-1]

# 3. 畫圖
top_n = 15
plt.title("Top 15 Feature Importances (Ensemble Consensus)")
plt.bar(range(top_n), avg_importances[indices[:top_n]], align="center", color='green')
plt.xticks(range(top_n), X_train.columns[indices[:top_n]], rotation=90)
plt.tight_layout()
plt.show()

AttributeError: 'XGBRegressor' object has no attribute 'named_estimators_'

<Figure size 1000x600 with 0 Axes>

## 5. 預測結果視覺化 (Visualization)

將模型對最後 30 天的預測結果與真實股價進行對比。
* **藍線**: 真實股價 (Actual)
* **紅虛線**: 模型預測 (Predicted)
* **觀察重點**: 紅線是否能跟上藍線的趨勢轉折？有沒有出現嚴重的滯後 (Lag)？

In [ ]:
# 畫圖檢查 - 預測 vs 真實 (看模型有沒有抓到趨勢)
preds = model.predict(X_val)

plt.figure(figsize=(12, 6))
# 畫出真實股價 (藍線)
plt.plot(val_df['date'], y_val, label='Actual Price', color='blue')
# 畫出預測股價 (紅虛線)
plt.plot(val_df['date'], preds, label='Predicted Price', color='red', linestyle='--')

plt.title("Prediction vs Actual (Last 30 Days)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. 特徵相關性分析 (Correlation Heatmap)

進一步分析前 15 大重要特徵與目標股價 (Target) 之間的相關性。
* **紅色 (正相關)**: 指標漲，股價跟著漲。
* **藍色 (負相關)**: 指標漲，股價反而跌。

In [ ]:

# --- 使用 Seaborn 畫熱力圖 ---
plt.figure(figsize=(12, 10))

# 1. 挑選要觀察的欄位 (這裡只選前 15 個重要特徵 + 股價)
# 假設 indices 是剛剛算出來的重要特徵排序
top_features = X_train.columns[indices[:15]].tolist()
plot_cols = top_features + [TARGET_COL]

# 2. 計算相關係數矩陣
corr_matrix = train_df[plot_cols].corr()

# 3. 畫熱力圖
plt.title("Feature Correlation Heatmap", fontsize=15)
sns.heatmap(
    corr_matrix, 
    annot=True,       # 顯示數字
    fmt=".2f",        # 數字格式 (小數點兩位)
    cmap="coolwarm",  # 顏色 (紅=正相關, 藍=負相關)
    linewidths=0.5,   # 格子邊框
    square=True
)
plt.show()

## 7. 最終評估結論 (Conclusion)

綜合評估模型在驗證集上的表現：
* **RMSE (均方根誤差)**: 預測值與真實值平均相差多少元。
* **MAPE (平均絕對百分比誤差)**: 預測誤差佔股價的百分比。

In [ ]:
# 1. 計算 RMSE
rmse = np.sqrt(mean_squared_error(y_val, preds))

# 2. 計算 MAPE
mape = mean_absolute_percentage_error(y_val, preds)

print(f"📉 集成模型 RMSE (誤差金額): {rmse:.4f}")
print(f"📊 集成模型 MAPE (誤差比例): {mape:.2%}")

# --- 簡單判讀 ---
print("-" * 30)
if mape < 0.05:
    print("🏆 評語：夢幻隊伍太強了！誤差不到 5%！")
elif mape < 0.10:
    print("✅ 評語：還不錯，多個專家投票果然穩，誤差在 10% 內。")
else:
    print("⚠️ 評語：好像沒有比單一模型準？可能要檢查是不是有哪位專家(例如隨機森林)在扯後腿。")

In [ ]:
import os

bad_file = 'experiments/history.csv'

# 檢查檔案在不在
if os.path.exists(bad_file):
    # 檢查是不是真的全空 (大小為 0)
    if os.path.getsize(bad_file) == 0:
        os.remove(bad_file)
        print(f"✅ 已刪除損壞的空檔案：{bad_file}")
        print("👉 現在請重新執行下面那一個 '寫入紀錄' 的 Cell，它會自動建立一個健康的檔案！")
    else:
        print("⚠️ 檔案存在且有內容，可能格式有錯？建議手動去左邊資料夾把它刪除。")
else:
    print("❓ 檔案本來就不在，您可以直接跑下面的程式碼。")

In [ ]:
# # ==========================================
# # 📝 實驗紀錄 (Experiment Logger)
# # ==========================================
# import experiment_logger

# # 1. 把您這次用的「參數」寫下來 (這很重要，不然以後會忘記這分數怎來的)
# # 如果是用集成模型：
# model_params = "Ensemble (XGB*2 + RF*1 + LGBM*1)"
# # 如果是用單一模型：
# # model_params = "XGBoost (n_estimators=1000, lr=0.05)"

# # 2. 寫入紀錄
# experiment_logger.log_experiment(
#     model_name="Ensemble_v1",      # 您可以自己取個代號
#     params=model_params,           # 參數說明
#     rmse=rmse,                     # 這是上面算出來的變數
#     mape=mape,                     # 這是上面算出來的變數
#     note="嘗試了集成模型，效果似乎比單一模型穩" # 寫給自己看的筆記
# )

# # 3. (選用) 看一下目前的排行榜
# print("\n🏆 歷史最佳紀錄：")
# try:
#     history = pd.read_csv('experiments/history.csv')
#     # 依照 MAPE 由小到大排 (越小越準)
#     print(history.sort_values('MAPE').head(3)[['Timestamp', 'Model', 'MAPE', 'Note']])
# except:
#     pass